In [68]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url = 'https://www.opentable.com/new-york-restaurant-listings'


driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [126]:
from selenium.webdriver.common.by import By

# Scraping list of all restaurants

The restaurants are listed on several different pages as opposed to one large infinite scroll. So I am fiding the number of total pages, then creating a loop that for the length of total pages, click next, and add to soup.

In [70]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [8]:
# this finds the total number of pages and saves the number as an integer for looping
total_pages = soup.find('div', class_= 'pagination-container').find_all('li')[-2].text
total_pages = int(total_pages.strip())

In [9]:
total_pages

100

In [11]:

#next_button = soup.find('a', href='fake url')

In [71]:
next_button = driver.find_element_by_xpath('//*[@id="results-pagination"]/li[9]/a')

In [72]:
next_button.click()

In [67]:
# was attempting to gather all html first to make master list, but seems you cannot add to BeautifulSoup
# going to make a list page by page, and add to that dictionary
soup = soup + BeautifulSoup(driver.page_source, 'html5lib')

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=90.0.4430.93)


# Creating list of restaurants

In [23]:
#this finds the table of all restaurants by page
table = soup.find('div', id= 'consumer-disclosure-tooltip').findNext()

In [24]:
# this is an individual row of each table
table.find('li', class_= 'result content-section-list-row cf with-times')

<li class="result content-section-list-row cf with-times" data-campaignids="7" data-id="0" data-index="1" data-is-marketed="true" data-is-promoted="true" data-lat="40.8294860" data-lon="-73.9804300" data-offers="" data-restaurant-availability-token="eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ" data-rid="217606"><div class="rest-row with-image"> <div class="rest-row-image"> <a href="https://www.opentable.com/r/villa-amalfi-cliffside-park?avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&amp;corrid=ef4d4920-3430-40f1-bf63-66d682887fa7" onclick="OT.BestAnalytics.logRestaurantVisit(217606)" tabindex="-1" target="_blank"> <img alt="photo of villa amalfi restaurant" class="lazy rest-image loaded" data-src="https://resizer.otstatic.com/v2/photos/legacy/3/31663170.jpg" src="https://resizer.otstatic.com/v2/photos/legacy/3/31663170.jpg"/> </a></div> <div class="rest-row-info"><div class="rest-row-header-container"><div class="rest-row-header flex-row-justify"> <a class="rest-row-name rest-name" href

In [29]:
#url for for the restaurant for this row
table.find('li', class_= 'result content-section-list-row cf with-times').a.get('href')

'https://www.opentable.com/r/villa-amalfi-cliffside-park?avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&corrid=ef4d4920-3430-40f1-bf63-66d682887fa7'

In [31]:
#name of restaurant.
table.find('span', class_= 'rest-row-name-text').text

'Villa Amalfi'

In [42]:
#number of times booked today 
num_bookings = int(table.find('div', class_='booking').text.split()[1])
num_bookings

3

In [32]:
rows = [row for row in table.find_all('li', class_= 'result content-section-list-row cf with-times')]
# rows[0]
len(rows)

100

In [63]:
big_dict = {}

In [61]:
mini_dict = {}

for row in rows:
    title = row.find('span', class_= 'rest-row-name-text').text
    url = row.find('a').get('href')
    if not row.find('div', class_='booking'):
        num_bookings = 0
    else:
        num_bookings = int(table.find('div', class_='booking').text.split()[1])
        
        
    mini_dict[title] = (url,
                       num_bookings)



In [64]:
big_dict.update(mini_dict)

In [75]:
big_dict

{'Villa Amalfi': ('https://www.opentable.com/r/villa-amalfi-cliffside-park?avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&corrid=ef4d4920-3430-40f1-bf63-66d682887fa7',
  3),
 'Bianco Blu': ('https://www.opentable.com/r/bianca-blu-edgewater?avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&corrid=ef4d4920-3430-40f1-bf63-66d682887fa7',
  3),
 'French Roast Uptown': ('https://www.opentable.com/r/french-roast-uptown-new-york?avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&corrid=ef4d4920-3430-40f1-bf63-66d682887fa7',
  3),
 'HAVEN Riverfront Restaurant and Bar': ('https://www.opentable.com/r/haven-riverfront-restaurant-and-bar-edgewater?avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&corrid=ef4d4920-3430-40f1-bf63-66d682887fa7',
  3),
 'The Leopard at des Artistes': ('https://www.opentable.com/r/the-leopard-at-des-artistes-new-york?avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&corrid=ef4d4920-3430-40f1-bf63-66d682887fa7',
  3),
 'Cafe Du Soleil': ('https://www.opentable.com/r/cafe-du-soleil-ne

# Putting it together into a loop

In [142]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url = 'https://www.opentable.com/new-york-restaurant-listings'


driver = webdriver.Chrome(chromedriver)
driver.get(url)




In [143]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [144]:
total_pages = soup.find('div', class_= 'pagination-container').find_all('li')[-2].text
total_pages = int(total_pages.strip())



In [145]:
big_dict = {}



In [151]:
#trying to loop till the last page of restaurants
for i in range(total_pages-1):
    soup = BeautifulSoup(driver.page_source, 'html5lib')

    rows = [row for row in table.find_all('li', class_= 'result content-section-list-row cf with-times')]
    
    mini_dict = {}

    for row in rows:
        title = row.find('span', class_= 'rest-row-name-text').text
        url = row.find('a').get('href')
        price_rating = len(row.find(class_='pricing--the-price').text.split())
        if not row.find('div', class_='booking'):
            num_bookings = 0
        else:
            num_bookings = int(table.find('div', class_='booking').text.split()[1])
        
        
        mini_dict[title] = (url,
                           num_bookings)
    big_dict.update(mini_dict)
    #next_button = driver.find_element_by_xpath('//*[@id="results-pagination"]/li[9]/a')
    #next_button = soup.find('a', href='fake url')
    next_button = driver.find_element_by_link_text('Next')
    next_button.click()


IndentationError: unexpected indent (<ipython-input-151-08ba1edc6da8>, line 24)

In [107]:
soup.find('a', href='fake url')

<a class="pagination-link pagination-prev" href="fake url"><span class="underline-hover">Previous</span></a>

In [148]:
driver.find_element_by_linkText('Next')

AttributeError: 'WebDriver' object has no attribute 'find_element_by_linkText'

In [128]:
next_button = driver.find_element(By.linkText('Next'))

AttributeError: type object 'By' has no attribute 'linkText'

In [125]:
driver.find_element_by_class_name('pagination-link pagination-next')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".pagination-link pagination-next"}
  (Session info: chrome=90.0.4430.93)


In [149]:
button = driver.find_element_by_link_text('Next')

In [132]:
button.click()

In [141]:
#decided to also pull price rater, the number of dollar signs from the first page.
len(soup.find(class_='pricing--the-price').text.split())

4